Vamos a listar las "jotas", es decir, el índice de las coordenadas totales, siguiendo este orden: de la esquina inferior [1,1], ese será j=1, luego j=2 corresponderá a [1,2], j=64 será [1,64] y luego j=65 será [2,1]. Recuerda que esto está ordenado como MATRICES, entonces j=5, [1,5]  es el lugar coordenada x=5 y y=1. Respeta el orden y todo saldrá bien.

In [1]:
ConjuntoDeCoordenadasTotal=Array[]
LasXNetas=Array[]
CasiXNetas=Array[]

0-element Array{Array{T,N},1}

Por cierto, estoy SEGURO que esta NO es la forma más inteligente de hacerlo, pero funciona. Ya después optimizamos.

In [2]:
for j=1:64,k=1:64
    push!(ConjuntoDeCoordenadasTotal,[j,k])
end

for j=1:24, k=1:24
    push!(LasXNetas,[j+5,k+23])
    push!(CasiXNetas,[j,k])
end

In [27]:
size(LasXNetas)

(576,)

In [3]:
BceroDura=readdlm("BceroDura.dat");
BceroSuave=readdlm("BceroSuave.dat")
btildedura=readdlm("BtildeDuraCero.dat")
btildesuave=readdlm("BtildeSuaveCero.dat");

In [4]:
BceroSuave[64,64]=393.105 #Convergencia, hay que checar el notebook anterior y ver porque pasa eso...

393.105

In [5]:
BceroDura[64,64]=137.089 #Correccion por convergencia, hay que checar el notebook anterior y ver porque pasa eso...

137.089

Dependiendo del cpu los procesos. En mi casa añado 5, en el labo añado 11.

In [6]:
addprocs(5)

5-element Array{Int64,1}:
 2
 3
 4
 5
 6

In [7]:
nprocs()

6

Esta parte de abajo debe ser trivialmente paralelizable, ¿no?

In [8]:
jmax=512
xconstante=[64,64]
Kduras=zeros(jmax,jmax);
#=
@time for j=1:jmax
     xj=ConjuntoDeCoordenadasTotal[j]
     for k=1:j
            xk=ConjuntoDeCoordenadasTotal[k]
        for l=1:jmax
            xl=ConjuntoDeCoordenadasTotal[l]
            Kduras[j,k]+=BceroDura[(xk-xl+xconstante)...]*BceroDura[(xj-xl+xconstante)...]
            end
    end
end
=#

512x512 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [9]:
@everywhere function sumaalgunasB(LasB::Array, lasX::Array,rango)
    tantas=length(rango)
    tamanho=length(lasX)
    result=zeros(tamanho,tamanho)
    xconstante=[64,64]
    for j=1:tamanho
     xj=lasX[j]
         for k=1:j
            xk=lasX[k]
            aux=0
            for l in rango
                xl=lasX[l]
                aux += LasB[(xk-xl+xconstante)...]*LasB[(xj-xl+xconstante)...]
          end
            result[j,k]=aux
  #  println("hola loco ", KdurasParalel[j,k])
        end
    end
    return result
    
end

@everywhere function sumaalgunasByBtilde(LasB::Array, LasBtilde::Array, lasX::Array,rango)
    tantas=length(rango)
    tamanho=length(lasX)
    result=zeros(tamanho,tamanho)
    xconstante=[64,64]
    for j=1:tamanho
     xj=lasX[j]
        for k=1:tamanho
            xk=lasX[k]
            aux=0
            for l in rango
                xl=lasX[l]
                aux += LasBtilde[(xk-xl+xconstante)...]*LasB[(xj-xl+xconstante)...]
          end
            result[j,k]=aux
  #  println("hola loco ", KdurasParalel[j,k])
        end
    end
    return result
    
end


In [10]:
@time sumaalgunasB(BceroDura, ConjuntoDeCoordenadasTotal, 1:2);

 45.151483 seconds (377.83 M allocations: 13.765 GB, 3.24% gc time)


In [11]:
@time sumaalgunasByBtilde(BceroDura,btildedura,ConjuntoDeCoordenadasTotal,1:2)

 91.762878 seconds (754.99 M allocations: 27.376 GB, 3.30% gc time)


4096x4096 Array{Float64,2}:
 272.827   272.827   272.827   272.827   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 272.827   272.827   272.827   272.827      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 267.424   267.424   267.424   267.424      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 256.597   256.597   256.597   256.597      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 240.273   240.273   240.273   240.273      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 218.366   218.366   218.366   218.366   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 193.743   193.743   193.743   193.743      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 171.641   171.641   171.641   171.641      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 153.754   153.754   153.754   153.754      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 139.009   139.009   139.009   139.009      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 126.673   126.673   126.673   126.673   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 116.226   116.226   116.226   116.226      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 107.283   107.283   107.283   107.2

In [12]:
@everywhere function divideArray(lasX::Array)
    #el proceso myid()==nprocs() va a ser el director de orquesta
    procesos=nprocs()-1
    largo=length(lasX)
    idx=myid()
    if idx==nprocs()
        return 1:0
    else
        splits=[round(Int, s) for s in linspace(0,largo,procesos+1)]
        return splits[idx]+1:splits[idx+1]
    end
end

In [13]:
@everywhere function obtenKParcial(LasB,lasX)
    rangito=divideArray(lasX)
    @show (rangito)
    result= sumaalgunasB(LasB, lasX,rangito)
    return result
end

@everywhere function obtenKtildeParcial(LasB,LasBTilde, lasX)
    rangito=divideArray(lasX)
    @show (rangito)
    result= sumaalgunasByBtilde(LasB, LasBTilde, lasX,rangito)
    return result
end


In [14]:
LasXchiquitas=ConjuntoDeCoordenadasTotal[1:512];
@time  laKParcial=obtenKParcial(BceroDura,LasXchiquitas)

rangito = 1:102
 34.778156 seconds (294.96 M allocations: 10.788 GB, 3.56% gc time)


512x512 Array{Float64,2}:
      3.01481e5      0.0        …      0.0            0.0      0.0
      3.18723e5      3.3755e5          0.0            0.0      0.0
      3.33315e5      3.53686e5         0.0            0.0      0.0
 344651.0            3.66436e5         0.0            0.0      0.0
      3.52181e5      3.75181e5         0.0            0.0      0.0
      3.55403e5      3.79363e5  …      0.0            0.0      0.0
      3.54713e5      3.79321e5         0.0            0.0      0.0
      3.51088e5      3.75996e5         0.0            0.0      0.0
 345350.0            3.70206e5         0.0            0.0      0.0
      3.38179e5      3.6266e5          0.0            0.0      0.0
      3.30123e5      3.5396e5   …      0.0            0.0      0.0
 321591.0            3.44596e5         0.0            0.0      0.0
      3.12865e5      3.34934e5         0.0            0.0      0.0
      ⋮                         ⋱                     ⋮           
  87233.7        91505.9        …   

In [15]:

@time  laKParcial=obtenKtildeParcial(BceroDura, btildedura,LasXchiquitas)

rangito = 1:102
 69.442915 seconds (588.51 M allocations: 21.522 GB, 3.57% gc time)


512x512 Array{Float64,2}:
 1385.79   1578.19   1748.93   1902.06   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1438.92   1655.92   1848.32   2019.06      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1462.03   1700.94   1917.94   2110.34      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1455.24   1710.49   1949.4    2166.4       0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1418.53   1684.6    1939.85   2178.76      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1351.71   1623.19   1889.26   2144.51   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1260.82   1532.29   1803.77   2069.84      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1155.93   1422.0    1693.47   1964.95      0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1045.94   1301.18   1567.25   1838.73      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  939.012  1177.92   1433.15   1699.22      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  842.767  1059.79   1298.69   1553.93   …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  761.14    953.987  1171.01   1409.91      0.0  0.0  0.0  0.0  0.0  0.0  0.0
  693.112   864.129  1056.98   1273.99

In [16]:
function sacalaKenTrozos(LasB::Array, lasX::Array)
    largo=length(lasX)
    cachos=nprocs()
    aux=SharedArray(Float64,(largo,largo,cachos))
     @sync begin
        for p in procs()
            @async  aux[:,:,p]=remotecall_fetch(p, obtenKParcial, LasB, lasX)
        end
        end #cierra el syncbegin
    result=zeros(largo,largo)
    result=sum(aux,3)
    return result
    #return result
end

function KtildeenTrozos(LasB::Array, LasBtilde::Array, lasX::Array)
    largo=length(lasX)
    cachos=nprocs()
    aux=SharedArray(Float64,(largo,largo,cachos))
     @sync begin
        for p in procs()
            @async  aux[:,:,p]=remotecall_fetch(p, obtenKtildeParcial, LasB, LasBtilde, lasX)
        end
        end #cierra el syncbegin
    result=zeros(largo,largo)
    result=sum(aux,3)
    return result
    #return result
end

KtildeenTrozos (generic function with 1 method)

In [17]:
@time K3R7=sacalaKenTrozos(BceroDura,LasXNetas);

rangito = 1:115
	From worker 4:	rangito = 347:461
	From worker 5:	rangito = 462:576
	From worker 2:	rangito = 116:230
	From worker 3:	rangito = 231:346
	From worker 6:	rangito = 1:0
 69.060873 seconds (422.02 M allocations: 15.465 GB, 3.16% gc time)


In [18]:
@time KTilde3R7=KtildeenTrozos(BceroDura,btildedura, CasiXNetas)

rangito = 1:115
	From worker 6:	rangito = 1:0
	From worker 4:	rangito = 347:461
	From worker 5:	rangito = 462:576
	From worker 2:	rangito = 116:230
	From worker 3:	rangito = 231:346
214.463882 seconds (839.76 M allocations: 30.732 GB, 1.83% gc time)


576x576x1 Array{Float64,3}:
[:, :, 1] =
 3132.23   3608.47   4032.9    4414.03   …   804.529   684.222   566.653
 3235.99   3770.23   4246.47   4670.9        823.393   700.319   580.011
 3269.02   3853.63   4387.87   4864.1        842.569   716.713   593.64 
 3232.29   3852.58   4437.18   4971.42       861.995   733.358   607.503
 3127.75   3769.17   4389.46   4974.06       881.594   750.193   621.556
 2961.18   3609.5    4250.91   4871.2    …   901.275   767.148   635.747
 2750.46   3391.71   4040.03   4681.44       920.928   784.137   650.01 
 2517.9    3138.29   3779.55   4427.86       940.43    801.06    664.269
 2282.77   2869.8    3490.2    4131.45       959.637   817.804   678.434
 2061.29   2604.18   3191.22   3811.62       978.39    834.237   692.404
 1865.88   2356.63   2899.52   3486.56   …   996.512   850.216   706.063
 1698.94   2138.73   2629.48   3172.37      1013.81    865.58    719.284
 1557.22   1952.26   2392.05   2882.79      1030.09    880.16    731.928
    ⋮      

In [22]:
KTilde3R7bien=KtildeenTrozos(BceroDura,btildedura, LasXNetas)

rangito = 1:115
	From worker 2:	rangito = 116:230
	From worker 3:	rangito = 231:346
	From worker 4:	rangito = 347:461
	From worker 6:	rangito = 1:0
	From worker 5:	rangito = 462:576


576x576x1 Array{Float64,3}:
[:, :, 1] =
 3132.23   3608.47   4032.9    4414.03   …   804.529   684.222   566.653
 3235.99   3770.23   4246.47   4670.9        823.393   700.319   580.011
 3269.02   3853.63   4387.87   4864.1        842.569   716.713   593.64 
 3232.29   3852.58   4437.18   4971.42       861.995   733.358   607.503
 3127.75   3769.17   4389.46   4974.06       881.594   750.193   621.556
 2961.18   3609.5    4250.91   4871.2    …   901.275   767.148   635.747
 2750.46   3391.71   4040.03   4681.44       920.928   784.137   650.01 
 2517.9    3138.29   3779.55   4427.86       940.43    801.06    664.269
 2282.77   2869.8    3490.2    4131.45       959.637   817.804   678.434
 2061.29   2604.18   3191.22   3811.62       978.39    834.237   692.404
 1865.88   2356.63   2899.52   3486.56   …   996.512   850.216   706.063
 1698.94   2138.73   2629.48   3172.37      1013.81    865.58    719.284
 1557.22   1952.26   2392.05   2882.79      1030.09    880.16    731.928
    ⋮      

In [23]:
KTilde3R7-KTilde3R7bien

576x576x1 Array{Float64,3}:
[:, :, 1] =
      1.37332e6  1.43345e6  1.48505e6  …  9.30673e5  896834.0      
      1.43382e6  1.49834e6  1.55424e6     9.65662e5       9.30777e5
      1.48582e6  1.55464e6  1.61503e6     9.99723e5       9.63836e5
      1.52732e6  1.60012e6  1.66497e6     1.03227e6  995442.0      
      1.55683e6  1.63309e6  1.70214e6     1.06276e6       1.02508e6
      1.57367e6  1.65271e6  1.72547e6  …  1.09078e6       1.05233e6
      1.57904e6  1.66003e6  1.7358e6      1.1162e6        1.07708e6
      1.57489e6  1.65693e6  1.73485e6     1.13896e6       1.0993e6 
      1.56299e6  1.64519e6  1.72433e6     1.15903e6       1.11893e6
      1.54488e6  1.62643e6  1.70587e6     1.17631e6       1.13588e6
      1.52184e6  1.60207e6  1.68096e6  …  1.1907e6        1.15007e6
      1.49489e6  1.57326e6  1.65091e6     1.20206e6       1.16135e6
      1.46477e6  1.54094e6  1.61682e6     1.21024e6       1.16958e6
      ⋮                                ⋱                  ⋮        
      1.

In [20]:
longus=size(K3R7,1)
K3R7=reshape(K3R7,longus,longus)
KTilde3R7=reshape(K3R7,longus,longus)
K3R7=K3R7+transpose(K3R7)

for j=1:longus
    KTilde3R7[j,j]=KTilde3R7[j,j]/2
end
writedlm("K3R7.dat",K3R7)
writedlm("KTilde3R7.dat",KTilde3R7)



In [25]:
KTilde3R7bien=reshape(KTilde3R7bien,longus,longus)
writedlm("KTilde3R7.dat",KTilde3R7bien)

In [ ]:
@everywhere function obtenerKernelParalell(LasB::Array, lasX::Array, jmax::Int)
    result=SharedArray(Float64,(jmax,jmax))
    @sync @parallel for j=1:jmax
      xj=lasX[j]
         for k=1:jmax
            xk=lasX[k]
            aux=0
          for l=1:jmax
                xl=lasX[l]
                aux = LasB[(xk-xl+xconstante)...]*LasB[(xj-xl+xconstante)...]
          end
            result[j,k]=aux
  #  println("hola loco ", KdurasParalel[j,k])
        end
    end
    result=Array(result)
    return result
end

@everywhere function obtenerKtildeParalell(LasB::Array,LasBTilde::Array, lasX::Array, jmax::Int)
    result=SharedArray(Float64,(jmax,jmax))
    @sync @parallel for j=1:jmax
        xj=lasX[j]
         for k=1:jmax
            xk=lasX[k]
         for l=1:jmax
                xl=lasX[l]
                result[j,k]=200
                unax=xk-xl+xconstante
                unay=xj-xl+xconstante
                if(LasBTilde[unax...]!=0.0)
                    println("hola tu")
                    
                #result[j,k]+= LasB[(xj-xl+xconstante)...]*LasBTilde[(xk-xl+xconstante)...]
                end
        end
  #  println("hola loco ", KdurasParalel[j,k])
        end
    end 
    return result
end
    

In [ ]:
#@time KSuaveParalel=obtenerKernelParalell(BceroSuave, ConjuntoDeCoordenadasTotal,4096);

In [ ]:
@time KDuraParalel2=obtenerKernelParalell(BceroDura, ConjuntoDeCoordenadasTotal,512);

In [ ]:
obtenerKtildeParalell(BceroDura,btildedura, ConjuntoDeCoordenadasTotal,128)

In [ ]:
writedlm("KSuavesParalel02.dat", KSuaveParalel)

In [ ]:
jmax=4096
KdurasParalel=SharedArray(Float64, jmax,jmax)

@time @sync @parallel for j=1:jmax
     xj=ConjuntoDeCoordenadasTotal[j]
     for k=1:j
            xk=ConjuntoDeCoordenadasTotal[k]

         for l=1:jmax
            xl=ConjuntoDeCoordenadasTotal[l]
            KdurasParalel[j,k]+= BceroDura[(xk-xl+xconstante)...]*BceroDura[(xj-xl+xconstante)...]
         
       
        end
  #  println("hola loco ", KdurasParalel[j,k])
    end
end 

In [ ]:
writedlm("KDurasParalel.dat", KdurasParalel)

In [ ]:
Kcasero=readdlm("/home/karel/Downloads/KDurasParalel.dat");

In [ ]:
Kcasero-KdurasParalel

In [ ]:
using PyPlot

In [ ]:
ion()
figuru=imshow(KSuaveParalel, origin="upper",interpolation="nearest",cmap="magma") 
cb=colorbar(figuru)

Hay algo raro... me está saliendo Ciclica la integral... ¿tiene sentido?

In [ ]:
figuru=imshow(KTotal, origin="upper",interpolation="nearest",cmap="magma") 
cb=colorbar(figuru)